In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np



2024-07-18 17:33:21.787836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=88000)

In [3]:
word_index = data.get_word_index() 

word_index = {k:(v+3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


def decode_review(text):
	return " ".join([reverse_word_index.get(i, "?") for i in text])

In [4]:
from keras.utils import pad_sequences

train_data = keras.utils.pad_sequences(train_data, value=word_index["<PAD>"], padding="post", maxlen=250)
test_data = keras.utils.pad_sequences(test_data, value=word_index["<PAD>"], padding="post", maxlen=250)

In [9]:
model = keras.Sequential()
model.add(keras.layers.Embedding(88000, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.summary
model.compile(optimizer="adam",loss="binary_crossentropy", metrics=["accuracy"])

In [10]:
x_val = train_data[:10000]
x_train = train_data[10000:]

y_val = train_labels[:10000]
y_train = train_labels[10000:]

In [23]:
y_val.dtype

dtype('int64')

In [22]:
test_labels.dtype

dtype('int64')

In [11]:
fitModel = model.fit(x_train, y_train, epochs=40, batch_size=512, validation_data=(x_val, y_val), verbose=1)
fitModel

Epoch 1/40
30/30 [==============================] - 3s 48ms/step - loss: 0.6923 - accuracy: 0.5169 - val_loss: 0.6908 - val_accuracy: 0.6524
Epoch 2/40
30/30 [==============================] - 1s 36ms/step - loss: 0.6882 - accuracy: 0.6405 - val_loss: 0.6850 - val_accuracy: 0.6990
Epoch 3/40
30/30 [==============================] - 1s 36ms/step - loss: 0.6788 - accuracy: 0.6927 - val_loss: 0.6727 - val_accuracy: 0.6994
Epoch 4/40
30/30 [==============================] - 1s 36ms/step - loss: 0.6606 - accuracy: 0.7361 - val_loss: 0.6514 - val_accuracy: 0.7282
Epoch 5/40
30/30 [==============================] - 1s 39ms/step - loss: 0.6313 - accuracy: 0.7779 - val_loss: 0.6205 - val_accuracy: 0.7557
Epoch 6/40
30/30 [==============================] - 1s 42ms/step - loss: 0.5916 - accuracy: 0.8043 - val_loss: 0.5821 - val_accuracy: 0.7919
Epoch 7/40
30/30 [==============================] - 1s 37ms/step - loss: 0.5447 - accuracy: 0.8329 - val_loss: 0.5409 - val_accuracy: 0.8002
Epoch 8/40
30

In [13]:
results = model.evaluate(test_data, test_labels)
results

782/782 [==============================] - 2s 3ms/step - loss: 0.3312 - accuracy: 0.8724


[0.3311576545238495, 0.872439980506897]

In [33]:
review=test_data[0]
predict=model.predict(test_data)
predict.dtype

782/782 [==============================] - 1s 2ms/step


dtype('float32')

In [37]:
Y_pred=[]
for i in predict:
    if i > 0.5:
        Y_pred.append(1)
    else:
        Y_pred.append(0)
Y_pred[:10]

[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]

In [38]:
model = keras.models.load_model("model.h5")

In [39]:
def review_encode(s):
	encoded = [1]

	for word in s:
		if word.lower() in word_index:
			encoded.append(word_index[word.lower()])
		else:
			encoded.append(2)

	return encoded
    

In [46]:
with open("/Users/icg/Downloads/movie_review.txt", encoding="utf-8") as f:
	for line in f.readlines():
		nline = line.replace(",", "").replace(".", "").replace("(", "").replace(")", "").replace(":", "").replace("\"","").strip().split(" ")
		encode = review_encode(nline)
		encode = keras.preprocessing.sequence.pad_sequences([encode], value=word_index["<PAD>"], padding="post", maxlen=250) # make the data 250 words long
		predict = model.predict(encode)
		print(line)
		print(encode)
Y_pred=[]
for i in predict:
    if i > 0.5:
        Y_pred.append(1)
    else:
        Y_pred.append(0)
print("Prediction:",Y_pred)




1/1 [==============================] - 0s 33ms/step
The best Disney animated film ever...This film had it all, it was funny, emotional, had family drama, and above all, great animation and songs! My personal favorite character is Rafiki, the Baboon! I still cant forget the line he says to Simbe, "Past can hurt, but as I see it, you can either run from it, or learn from it!" it is so true! i loved Zazu's weirdness, and shenzi's humor, above all, i loved Pumba's innocence and Timon's intelligence! In short, i found the film the best film ever... The voice cast is also great. Mathew did complete justice to Simba, and what can I say about Mufasa, He is the best King, and dad anyone can ever get! The movie is not meant for kids, its meant to teach every adult a lesson...to find our place in the great "Circle Of Life". I give this film a perfect 10.

[[    1     4   118   910  1125    22     2    22    69    12    32    12
     16   163   921    69   223   453     5   752    32    87   748  